In [4]:
# import pyspark
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType
# from pyspark.sql.types import ArrayType, DoubleType, BooleanType
# from pyspark.sql.functions import col,array_contains

se importa SparkSQL

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('EDA').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/27 18:40:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/27 18:40:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/27 18:40:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Se importan los documentos .csv necesarios para realizar el ejercicio y se leen los campos de cabecera, lo cual indica que contiene columnas con valores nulos

In [4]:
dataframe = spark.read.csv("./data/SII_roles_avaluo_table/cod_comunas.csv")
dataframe.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [5]:
dataframe2 = spark.read.option("header",True) \
              .csv("./data/SII_roles_avaluo_table/cod_comunas.csv")
dataframe2.printSchema()

root
 |-- comunaID: string (nullable = true)
 |-- region: string (nullable = true)
 |-- comuna: string (nullable = true)
 |-- n_region: string (nullable = true)
 |-- name_region: string (nullable = true)



se realiza conversión de documento .csv a tablas (de acuerdo al caracter delimitado por ,) para facilitar el trabajo. Se realiza esto por cada tabla y se imprimen las tablas para visualizar los valores de ellas

In [5]:
dataframe3 = spark.read.options(header='True', delimiter=',') \
              .csv("./data/SII_roles_avaluo_table/cod_comunas.csv")
dataframe3.printSchema()

root
 |-- comunaID: string (nullable = true)
 |-- region: string (nullable = true)
 |-- comuna: string (nullable = true)
 |-- n_region: string (nullable = true)
 |-- name_region: string (nullable = true)



In [7]:
spark.sql("SELECT * FROM cod_comunas;").show()

+--------+--------------------+--------------------+--------+--------------------+
|comunaID|              region|              comuna|n_region|         name_region|
+--------+--------------------+--------------------+--------+--------------------+
|    1101| 18_DIRECCION_REG...|               ARICA|      18| Regi�n de Arica ...|
|    1106| 18_DIRECCION_REG...|           CAMARONES|      18| Regi�n de Arica ...|
|    1302| 18_DIRECCION_REG...|       GENERAL LAGOS|      18| Regi�n de Arica ...|
|    1301| 18_DIRECCION_REG...|               PUTRE|      18| Regi�n de Arica ...|
|    1211| 1_DIRECCION_REGI...|       ALTO HOSPICIO|       1|  Regi�n de Tarapac�|
|    1208| 1_DIRECCION_REGI...|              CAMI�A|       1|  Regi�n de Tarapac�|
|    1210| 1_DIRECCION_REGI...|            COLCHANE|       1|  Regi�n de Tarapac�|
|    1206| 1_DIRECCION_REGI...|               HUARA|       1|  Regi�n de Tarapac�|
|    1201| 1_DIRECCION_REGI...|             IQUIQUE|       1|  Regi�n de Tarapac�|
|   

se limpian los datos eliminando la primera columna para facilitar la generación de las queries

In [8]:
def createTempTablefromCSV(csv_path, table_temp_name):
    dataframe = spark.read.options(header='True', delimiter=',', inferSchema= True) \
              .csv(csv_path)
    dataframe = dataframe.drop("_c0")
    dataframe.printSchema()
    dataframe.createOrReplaceTempView(table_temp_name)
    return

In [9]:
createTempTablefromCSV(
    csv_path="./data/SII_roles_avaluo_table/cod_comunas.csv",
    table_temp_name="cod_comunas")

root
 |-- comunaID: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- comuna: string (nullable = true)
 |-- n_region: string (nullable = true)
 |-- name_region: string (nullable = true)



In [10]:
createTempTablefromCSV(
    csv_path="./data/SII_roles_avaluo_table/rol_agricola.csv",
    table_temp_name="rol_agricola")

root
 |-- cod_comuna: integer (nullable = true)
 |-- nr_manzana: integer (nullable = true)
 |-- nr_predial: integer (nullable = true)
 |-- dir/nomb_predio: string (nullable = true)
 |-- avaluo_fisc_tot: long (nullable = true)
 |-- contri_semestral: integer (nullable = true)
 |-- cod_dest_princ: string (nullable = true)
 |-- avaluo_exen_prop: long (nullable = true)



In [11]:
createTempTablefromCSV(
    csv_path="./data/SII_roles_avaluo_table/rol_no_agricola.csv",
    table_temp_name="rol_no_agricola")

root
 |-- cod_comuna: integer (nullable = true)
 |-- nr_manzana: integer (nullable = true)
 |-- nr_predial: integer (nullable = true)
 |-- dir/nomb_predio: string (nullable = true)
 |-- avaluo_fisc_tot: long (nullable = true)
 |-- contri_semestral: integer (nullable = true)
 |-- cod_dest_princ: string (nullable = true)
 |-- avaluo_exen_prop: long (nullable = true)
 |-- cod_comuna_rol_bien_comun_1: integer (nullable = true)
 |-- nro_manzana_rol_bien_comun_1: integer (nullable = true)
 |-- nr_predio_rol_bien_comun_1: integer (nullable = true)
 |-- cod_comuna_rol_bien_comun_2: integer (nullable = true)
 |-- nro_manzana_rol_bien_comun_2: integer (nullable = true)
 |-- nr_predio_rol_bien_comun_2: integer (nullable = true)
 |-- sup_total_terreno_prod_m2: integer (nullable = true)



In [33]:
createTempTablefromCSV(
    csv_path="./data/SII_roles_avaluo_table/terreno_constr_no_agri.csv",
    table_temp_name="terreno_constr_no_agri")

root
 |-- cod_comuna: integer (nullable = true)
 |-- nr_manzana: integer (nullable = true)
 |-- nr_predial: integer (nullable = true)
 |-- corr_lin_prod: integer (nullable = true)
 |-- cod_material_estruc_const: string (nullable = true)
 |-- cod_calidad_linea_const: integer (nullable = true)
 |-- anio_linea_constr: integer (nullable = true)
 |-- sup_linea_constr_m2/m3: integer (nullable = true)
 |-- cod_dest_linea_constr: string (nullable = true)
 |-- cod_cond_especial_linea_constr: string (nullable = true)



In [36]:
spark.sql("SELECT * FROM rol_agricola;").show()

+----------+----------+----------+--------------------+---------------+----------------+--------------+----------------+
|cod_comuna|nr_manzana|nr_predial|     dir/nomb_predio|avaluo_fisc_tot|contri_semestral|cod_dest_princ|avaluo_exen_prop|
+----------+----------+----------+--------------------+---------------+----------------+--------------+----------------+
|      1101|       675|         6|LINEA FFCC TAC   ...|       14175739|               0|             A|        14175739|
|      1101|       675|        10|SANTA ROSA       ...|     1109356342|         2786914|             A|        24954042|
|      1101|       676|         6|CHINCHORRO       ...|      130704724|          271780|             A|        24954042|
|      1101|      1001|         6|CERROS LLUTA LTD ...|      345514803|          823842|             B|        24954042|
|      1101|      1001|        39|QUEBRADA ENCANTAD...|         164694|               0|             A|          164694|
|      1101|      1001|        4

# QUERY UNO
Por cada comuna sumar el total de las contribuciones semestrles (aquellas contribuciones mayores a 0 y el rol agrícola sea A). 
Se agrupa por tipo de región y se muestra de mayor a menor contribución semestral

In [87]:
SQL = "SELECT cod_comunas.name_region, cod_comunas.comuna,  SUM(rol_agricola.contri_semestral) AS total_contrib_semestral " \
      "FROM cod_comunas INNER JOIN  rol_agricola ON cod_comunas.comunaID = rol_agricola.cod_comuna " \
      "WHERE (rol_agricola.contri_semestral > 0) AND (rol_agricola.cod_dest_princ = 'A') " \
      "GROUP BY cod_comunas.name_region, cod_comunas.comuna ORDER BY total_contrib_semestral DESC "\
      ";"

spark.sql(SQL).show()

+--------------------+-------------+-----------------------+
|         name_region|       comuna|total_contrib_semestral|
+--------------------+-------------+-----------------------+
| Regi�n Metropoli...|    MELIPILLA|              841081258|
|  Regi�n de Los R�os|    R�O BUENO|              701278902|
|   Regi�n del Biob�o|  LOS ANGELES|              683634258|
| Regi�n Metropoli...|        PAINE|              596352680|
| Regi�n de Los Lagos|       OSORNO|              577156140|
|    Regi�n del Maule| SAN CLEMENTE|              538904142|
|   Regi�n del Biob�o|      MULCH�N|              498592314|
| Regi�n Metropoli...|         BUIN|              494186870|
| Regi�n de La Ara...|     VICTORIA|              445634720|
| Regi�n del Liber...|  CHIMBARONGO|              441050924|
|  Regi�n de Los R�os|     LA UNI�N|              419097364|
| Regi�n de Los Lagos|    R�O NEGRO|              417473524|
| Regi�n de Los Lagos|    PURRANQUE|              414030320|
|     Regi�n de �uble|  

# QUERY DOS
seleccionar las comunas de la segunda región y sumar el total de las contribuciones de los roles agrícolas, mostrando en orden descendente y también el nombre de la región

In [99]:
SQL = "SELECT cod_comunas.name_region, cod_comunas.comuna,  SUM(rol_agricola.contri_semestral) AS total_contrib_semestral " \
      "FROM cod_comunas INNER JOIN  rol_agricola ON cod_comunas.comunaID = rol_agricola.cod_comuna " \
      "WHERE (cod_comunas.n_region = 2)" \
      "GROUP BY cod_comunas.name_region, cod_comunas.comuna ORDER BY total_contrib_semestral DESC "\
      ";"
spark.sql(SQL).show()

+--------------------+--------------------+-----------------------+
|         name_region|              comuna|total_contrib_semestral|
+--------------------+--------------------+-----------------------+
| Regi�n de Antofa...| SAN PEDRO DE ATA...|                8778278|
| Regi�n de Antofa...|         ANTOFAGASTA|                5272548|
| Regi�n de Antofa...|              TALTAL|                5007746|
| Regi�n de Antofa...|              CALAMA|                4307122|
| Regi�n de Antofa...|         MAR�A ELENA|                 121000|
| Regi�n de Antofa...|        SIERRA GORDA|                  21254|
| Regi�n de Antofa...|          MEJILLONES|                      0|
| Regi�n de Antofa...|           TOCOPILLA|                      0|
| Regi�n de Antofa...|             OLLAG�E|                      0|
+--------------------+--------------------+-----------------------+



# QUERY TRES
lo mismo de la query dos, pero para los roles no agrícolas de la segunda región


In [100]:
SQL = "SELECT cod_comunas.name_region, cod_comunas.comuna,  SUM(rol_no_agricola.contri_semestral) AS total_contrib_semestral " \
      "FROM cod_comunas INNER JOIN  rol_no_agricola ON cod_comunas.comunaID = rol_no_agricola.cod_comuna " \
      "WHERE (cod_comunas.n_region = 2)" \
      "GROUP BY cod_comunas.name_region, cod_comunas.comuna ORDER BY total_contrib_semestral DESC "\
      ";"
spark.sql(SQL).show()

+--------------------+--------------------+-----------------------+
|         name_region|              comuna|total_contrib_semestral|
+--------------------+--------------------+-----------------------+
| Regi�n de Antofa...|         ANTOFAGASTA|            19369774342|
| Regi�n de Antofa...|              CALAMA|             7252880618|
| Regi�n de Antofa...|          MEJILLONES|             1253149696|
| Regi�n de Antofa...|        SIERRA GORDA|              558217478|
| Regi�n de Antofa...|           TOCOPILLA|              447653712|
| Regi�n de Antofa...|         MAR�A ELENA|              308896368|
| Regi�n de Antofa...| SAN PEDRO DE ATA...|              231002214|
| Regi�n de Antofa...|              TALTAL|               86723788|
| Regi�n de Antofa...|             OLLAG�E|                5028532|
+--------------------+--------------------+-----------------------+

